In [1]:
import requests
import pandas as pd
from datetime import datetime

In [2]:
def convert_date(date_str):
    """
    날짜 문자열을 datetime으로 변환하는 함수
    유효하지 않은 날짜(9999년 등)는 None으로 처리
    """
    try:
        if date_str == '99991231':
            return None
        return pd.to_datetime(date_str, format='%Y%m%d')
    except:
        return None

In [4]:
def get_credit_loan_products(api_key):
    """
    금융감독원 개인신용대출 상품 정보를 가져와서 DataFrame으로 변환하는 함수

    Parameters:
        api_key (str): 금융감독원 API 인증키

    Returns:
        pandas.DataFrame: 대출상품 정보가 담긴 DataFrame
    """

    # API 엔드포인트 설정
    base_url = "http://finlife.fss.or.kr/finlifeapi/creditLoanProductsSearch.json"

    # API 요청 파라미터 설정
    params = {
        "auth": api_key,
        "topFinGrpNo": "020000",  # 은행권
        "pageNo": "1"
    }

    try:
        # API 요청
        response = requests.get(base_url, params=params)
        response.raise_for_status()  # HTTP 에러 체크

        # JSON 데이터 추출
        data = response.json()

        # 대출상품 목록 추출
        products = data.get('result', {}).get('baseList', [])

        # DataFrame 생성
        df = pd.DataFrame(products)

        # 컬럼명 한글로 변경 (필요한 컬럼만 선택)
        column_mapping = {
            'fin_co_no': '금융회사코드',
            'kor_co_nm': '금융회사명',
            'fin_prdt_cd': '금융상품코드',
            'fin_prdt_nm': '금융상품명',
            'join_way': '가입방법',
            'loan_inci_expn': '대출부대비용',
            'erly_rpay_fee': '중도상환수수료',
            'dcls_strt_day': '공시시작일',
            'dcls_end_day': '공시종료일'
        }

        df = df.rename(columns=column_mapping)

        # 날짜 데이터 처리
        df['공시시작일'] = df['공시시작일'].apply(convert_date)
        df['공시종료일'] = df['공시종료일'].apply(convert_date)

        # 현재 유효한 상품만 필터링 (선택사항)
        current_date = pd.Timestamp.now()
        df['현재유효여부'] = (
            (df['공시시작일'] <= current_date) &
            ((df['공시종료일'].isna()) | (df['공시종료일'] >= current_date))
        )

        # 데이터 출력 형식 지정
        pd.set_option('display.max_columns', None)
        pd.set_option('display.width', None)

        return df

    except requests.exceptions.RequestException as e:
        print(f"API 요청 중 오류 발생: {e}")
        return None
    except Exception as e:
        print(f"데이터 처리 중 오류 발생: {e}")
        return None

In [7]:
API_KEY = ""  # 발급받은 API 키를 입력하세요

df = get_credit_loan_products(API_KEY)

if df is not None:
    print("데이터 수집 완료")
    print(f"수집된 상품 수: {len(df)}")
    print("\n현재 유효한 상품 수:", df['현재유효여부'].sum())
    df

데이터 수집 완료
수집된 상품 수: 44

현재 유효한 상품 수: 44


In [6]:
df

,dcls_month,금융회사코드,금융상품코드,crdt_prdt_type,금융회사명,금융상품명,가입방법,cb_name,crdt_prdt_type_nm,공시시작일,공시종료일,fin_co_subm_day,현재유효여부
0,202410,0010001,CR0001A,1,우리은행,개인신용대출 우대고객,"영업점,인터넷,스마트폰",KCB,일반신용대출,2024-10-21,NaT,202410210900,True
1,202410,0010001,CR0001C,2,우리은행,개인신용대출 퍼펙트,"영업점,인터넷,스마트폰",KCB,마이너스한도대출,2024-10-21,NaT,202410210900,True
2,202410,0010002,SC001217,1,한국스탠다드차타드은행,개인신용대출,"영업점,스마트폰",KCB,일반신용대출,2024-10-18,NaT,202410181734,True
3,202410,0010002,SC001217_1,2,한국스탠다드차타드은행,개인신용대출,"영업점,스마트폰",KCB,마이너스한도대출,2024-10-18,NaT,202410181734,True
4,202410,0010006,WR0002F,3,한국씨티은행,장기카드대출,"인터넷,스마트폰,전화(텔레뱅킹),기타",KCB,장기카드대출(카드론),2024-10-21,NaT,202410111504,True
5,202410,0010016,WR0001F,1,아이엠뱅크,일반신용대출,"영업점,스마트폰",KCB,일반신용대출,2024-10-20,2024-11-30,202410170949,True
6,202410,0010016,WR0002F,2,아이엠뱅크,마이너스한도대출,"영업점,스마트폰",KCB,마이너스한도대출,2024-10-20,2024-11-30,202410170949,True
7,202410,0010016,WR0002F,3,아이엠뱅크,장기카드대출,"스마트폰,모집인",NICE,장기카드대출(카드론),2024-10-21,NaT,202410141350,True
8,202410,0010017,1300,1,부산은행,ONE신용대출,"영업점,스마트폰",KCB,일반신용대출,2024-10-14,NaT,202410141550,True
9,202410,0010017,1301,2,부산은행,ONE신용대출,"영업점,스마트폰",KCB,마이너스한도대출,2024-10-14,NaT,202410141550,True
